# Modelo de programación lineal

## Entrada de datos: Instancia 49x1

In [1]:
import pandas as pd

In [2]:
datafile="49x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

## Modelo

In [3]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
indices=[(i,j,k) for i in N for j in M for k in N]

HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

#Tiempo máximo de respuesta para pruebas: 1 hora.
mdl.Params.timeLimit=3600.0

mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")

mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')

mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])

for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xb532375f
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 87044 rows and 13308 columns
Presolve time: 2.22s
Presolved: 1401 rows, 1098 columns, 26878 nonzeros
Variable types: 0 continuous, 1098 integer (1098 binary)

Root relaxation: objective 4.900000e+01, 346 iterations, 0.05 seconds



## Entrada de datos: Instancia 49x2

In [4]:
datafile="49x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [5]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x79ddd645
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 86776 rows and 13268 columns
Presolve time: 1.59s
Presolved: 1669 rows, 1138 columns, 32326 nonzeros
Variable types: 0 continuous, 1138 integer (1138 binary)

Root relaxation: objective 4.900000e+01, 283 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x3

In [6]:
datafile="49x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [7]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x681f07df
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 87456 rows and 13416 columns
Presolve time: 2.20s
Presolved: 989 rows, 990 columns, 18501 nonzeros
Found heuristic solution: objective 48.0000000
Variable types: 0 continuous, 990 integer (990 binary)

Root relaxation: objective 4.900000e+01, 326 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 

## ## Entrada de datos: Instancia 49x4

In [8]:
datafile="49x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [9]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xe241cb44
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 87394 rows and 13396 columns
Presolve time: 1.95s
Presolved: 1051 rows, 1010 columns, 19929 nonzeros
Variable types: 0 continuous, 1010 integer (1010 binary)

Root relaxation: objective 4.900000e+01, 438 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x5

In [10]:
datafile="49x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [11]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x095c20b8
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 87529 rows and 13420 columns
Presolve time: 1.89s
Presolved: 916 rows, 986 columns, 17812 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 986 integer (986 binary)

Root relaxation: cutoff, 436 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

## Entrada de datos: Instancia 49x6

In [12]:
datafile="49x6.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [13]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xf7ba99fe
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 87385 rows and 13362 columns
Presolve time: 1.72s
Presolved: 1060 rows, 1044 columns, 21962 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 1044 integer (1044 binary)

Root relaxation: cutoff, 306 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 

## Entrada de datos: Instancia 49x7

In [14]:
datafile="49x7.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [15]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x1a0e0d52
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 86972 rows and 13261 columns
Presolve time: 2.09s
Presolved: 1473 rows, 1145 columns, 24512 nonzeros
Variable types: 0 continuous, 1145 integer (1143 binary)

Root relaxation: objective 4.900000e+01, 343 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x8

In [16]:
datafile="49x8.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [17]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xbf2a42e7
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 87289 rows and 13240 columns
Presolve time: 1.55s
Presolved: 1156 rows, 1166 columns, 23203 nonzeros
Variable types: 0 continuous, 1166 integer (1166 binary)

Root relaxation: objective 4.900000e+01, 276 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x9

In [18]:
datafile="49x9.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [19]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xc778bfb6
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 86918 rows and 13246 columns
Presolve time: 2.13s
Presolved: 1527 rows, 1160 columns, 30616 nonzeros
Variable types: 0 continuous, 1160 integer (1160 binary)

Root relaxation: objective 4.900000e+01, 233 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x10

In [20]:
datafile="49x10.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [21]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x0d88fccb
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 87365 rows and 13367 columns
Presolve time: 1.79s
Presolved: 1080 rows, 1039 columns, 20136 nonzeros
Variable types: 0 continuous, 1039 integer (1039 binary)

Root relaxation: objective 4.900000e+01, 361 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x11

In [22]:
datafile="49x11.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [23]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xbd5bdd2d
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 86635 rows and 13262 columns
Presolve time: 1.99s
Presolved: 1810 rows, 1144 columns, 32810 nonzeros
Variable types: 0 continuous, 1144 integer (1144 binary)

Root relaxation: objective 4.900000e+01, 321 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x12

In [24]:
datafile="49x12.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [25]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x2f63d795
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 86678 rows and 13264 columns
Presolve time: 1.94s
Presolved: 1767 rows, 1142 columns, 31128 nonzeros
Variable types: 0 continuous, 1142 integer (1142 binary)

Root relaxation: objective 4.900000e+01, 344 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x13

In [26]:
datafile="49x13.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [27]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x32de377b
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 87521 rows and 13179 columns
Presolve time: 0.78s
Presolved: 924 rows, 1227 columns, 17816 nonzeros
Variable types: 0 continuous, 1227 integer (1227 binary)

Root relaxation: objective 4.900000e+01, 315 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 49x14

In [28]:
datafile="49x14.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [29]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x1df380d1
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 87244 rows and 13348 columns
Presolve time: 1.75s
Presolved: 1201 rows, 1058 columns, 21276 nonzeros
Variable types: 0 continuous, 1058 integer (1058 binary)

Root relaxation: objective 4.900000e+01, 440 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x15

In [30]:
datafile="49x15.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [31]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x3d08fb44
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 87676 rows and 13230 columns
Presolve time: 0.58s
Presolved: 769 rows, 1176 columns, 15872 nonzeros
Variable types: 0 continuous, 1176 integer (1176 binary)

Root relaxation: objective 4.900000e+01, 270 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x16

In [32]:
datafile="49x16.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [33]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xbb8a110e
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 87067 rows and 13226 columns
Presolve time: 1.92s
Presolved: 1378 rows, 1180 columns, 25135 nonzeros
Variable types: 0 continuous, 1180 integer (1180 binary)

Root relaxation: objective 4.900000e+01, 358 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x17

In [34]:
datafile="49x17.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [35]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xc7bb5459
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 86882 rows and 13298 columns
Presolve time: 2.04s
Presolved: 1563 rows, 1108 columns, 30070 nonzeros
Variable types: 0 continuous, 1108 integer (1108 binary)

Root relaxation: objective 4.900000e+01, 320 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x18

In [36]:
datafile="49x18.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [37]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xd315a26d
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 86904 rows and 13224 columns
Presolve time: 2.06s
Presolved: 1541 rows, 1182 columns, 25728 nonzeros
Variable types: 0 continuous, 1182 integer (1182 binary)

Root relaxation: objective 4.900000e+01, 373 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x19

In [38]:
datafile="49x19.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [39]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xcd453252
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 87002 rows and 13340 columns
Presolve time: 1.87s
Presolved: 1443 rows, 1066 columns, 26070 nonzeros
Variable types: 0 continuous, 1066 integer (1066 binary)

Root relaxation: objective 4.900000e+01, 271 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x20

In [40]:
datafile="49x20.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [41]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x168c2680
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 87259 rows and 13372 columns
Presolve time: 1.75s
Presolved: 1186 rows, 1034 columns, 21812 nonzeros
Found heuristic solution: objective 48.0000000
Variable types: 0 continuous, 1034 integer (1034 binary)

Root relaxation: objective 4.900000e+01, 226 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

## Entrada de datos: Instancia 49x21

In [42]:
datafile="49x21.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [43]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x1ee46405
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 87133 rows and 13318 columns
Presolve time: 2.07s
Presolved: 1312 rows, 1088 columns, 23144 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 1088 integer (1088 binary)

Root relaxation: cutoff, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

## Entrada de datos: Instancia 49x22

In [44]:
datafile="49x22.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [45]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xc137d629
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 87456 rows and 13278 columns
Presolve time: 0.96s
Presolved: 989 rows, 1128 columns, 17384 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 4.900000e+01, 261 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x23

In [46]:
datafile="49x23.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [47]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x87c0846a
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 86990 rows and 13230 columns
Presolve time: 2.04s
Presolved: 1455 rows, 1176 columns, 25921 nonzeros
Variable types: 0 continuous, 1176 integer (1176 binary)

Root relaxation: objective 4.900000e+01, 235 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x24

In [48]:
datafile="49x24.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [49]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x0756cf3d
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 86770 rows and 13160 columns
Presolve time: 1.78s
Presolved: 1675 rows, 1246 columns, 27741 nonzeros
Variable types: 0 continuous, 1246 integer (1244 binary)

Root relaxation: objective 4.900000e+01, 365 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x25

In [50]:
datafile="49x25.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [51]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xcb29ede1
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 86780 rows and 13148 columns
Presolve time: 2.55s
Presolved: 1665 rows, 1258 columns, 33986 nonzeros
Variable types: 0 continuous, 1258 integer (1258 binary)

Root relaxation: objective 4.900000e+01, 269 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 49x26

In [52]:
datafile="49x26.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [53]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xe25f9d87
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 87257 rows and 13190 columns
Presolve time: 1.63s
Presolved: 1188 rows, 1216 columns, 25324 nonzeros
Variable types: 0 continuous, 1216 integer (1216 binary)

Root relaxation: objective 4.900000e+01, 245 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x27

In [54]:
datafile="49x27.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [55]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xfdf5fc9f
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 87080 rows and 13346 columns
Presolve time: 2.05s
Presolved: 1365 rows, 1060 columns, 23061 nonzeros
Variable types: 0 continuous, 1060 integer (1058 binary)

Root relaxation: objective 4.900000e+01, 338 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x28

In [56]:
datafile="49x28.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [57]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    


ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x3bef341d
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 86916 rows and 13137 columns
Presolve time: 2.42s
Presolved: 1529 rows, 1269 columns, 28613 nonzeros
Variable types: 0 continuous, 1269 integer (1269 binary)

Root relaxation: objective 4.900000e+01, 234 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x29

In [58]:
datafile="49x29.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [59]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0xfb550aa9
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 87294 rows and 13148 columns
Presolve time: 0.88s
Presolved: 1151 rows, 1258 columns, 23094 nonzeros
Variable types: 0 continuous, 1258 integer (1258 binary)

Root relaxation: objective 4.900000e+01, 343 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

## Entrada de datos: Instancia 49x30

In [60]:
datafile="49x30.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

49

In [61]:

p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 2
N = range(ith)
M = range(jth)
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

# Importar Gurobi
import gurobipy as grb

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 88445 rows, 14406 columns and 560266 nonzeros
Model fingerprint: 0x76a8d3c4
Variable types: 0 continuous, 14406 integer (14406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 86943 rows and 13258 columns
Presolve time: 1.25s
Presolved: 1502 rows, 1148 columns, 28832 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 1148 integer (1148 binary)

Root relaxation: cutoff, 118 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 